In [ ]:
!pip3 install pandarallel

In [ ]:
from pandarallel import pandarallel
import re
import string
import sys
import html
import pandas as pd
import json
import csv
import glob
import os.path
import time
import nltk
import logging as lg
from datetime import datetime as dt, timedelta
from gensim.summarization import keywords
from random import *
import numpy as np
from gensim.utils import simple_preprocess

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#drive.flush_and_unmount()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pandarallel.initialize()

In [ ]:
df = pd.read_csv("/content/drive/My Drive/datasets/dataset_processed.csv",engine="python")
df["subreddit"] = df["subreddit"].str.lower()
df["text"] = df["text"].apply(lambda x: '\n'.join(preprocess(x.splitlines())))
print(df.info)
df.head()

In [ ]:
subs_arr = ["unpopularopinion","explainlikeimfive","oldschoolcool","aww","funny","politics","politicalhumor","politicalcompassMemes","conservative",
            "therightcantmeme","neoliberal","democrats","politicaldiscussion",
            "republicans", "libertarian","askreddit","showerthoughts",
           "askscience","economics","askeconomics","badeconomics","programmerhumor",
            "interestingasfuck","askwomen"]
            
pol_arr = ["politics","politicalhumor","politicalcompassMemes","conservative",
            "therightcantmeme","neoliberal","democrats","politicaldiscussion",
            "republicans", "libertarian"]

In [ ]:
punc = '''!()[]|{};":\<>/@#$%^&*_~'''

relp = " "*len(punc)
def preprocess(s):
  result = re.sub(r"’", "'", str(s))
  result = re.sub(r"\n", ' ', str(result))
  result = re.sub(r"r/", ' ', str(result))
  result = result.encode("ascii", "ignore").decode()
  result = re.sub(r"http\S+", ' ', str(result), flags=re.MULTILINE)
  result = html.unescape(result)
  result = result.translate(str.maketrans(punc,relp))
  result = re.sub(' +', ' ',str(result))
  result = result.strip()
  return result;

In [ ]:
def keywords_tr(x):
  kws = keywords(x,ratio=1,lemmatize=True,scores=True)[:10]
  return json.dumps(dict(kws))

TextRank is ridiculously slow. And I can't find any gpu implementations of it. So processing in chunks using multiprocessing.

In [ ]:
def split_dataframe(df, chunk_size = 5000): 
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

In [ ]:
i = 0
for df_chunk in split_dataframe(df,chunk_size=1000):
  print(i)
  df_chunk["text"]=df_chunk["text"].parallel_apply(lambda x: keywords_tr(x))
  df_chunk.to_csv(f"/content/drive/My Drive/datasets/keywords/{i}.csv",index=False)
  i+=1

In [ ]:
path = r'/content/drive/My Drive/datasets/keywords/' 
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
dff = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
dff["keywords"] = dff["text"].apply(lambda x: list(json.loads(x).keys()))
dff["scores"] = dff["text"].apply(lambda x: list(json.loads(x).values()))
dff["label"]=dff["subreddit"].apply(lambda x:x in pol_arr)
dff.drop(columns=["text"],inplace=True)
dff.info()

In [ ]:
dff.to_csv("/content/drive/My Drive/datasets/keywords.csv",index=False)

In [ ]:
def flatten_lst(lst):
  ret_list = []
  for i in lst:
    ret_list.extend(i)
  return ret_list

def split_words(lst):
  ret_list = []
  for i in lst:
    ret_list.extend(i.split())
  return ret_list

Extract Individual Words

In [ ]:
df["keywords"]=df["keywords"].apply(split_words)
pol_lst = flatten_lst(list(df[df["label"]==1]["keywords"]))
df_pol = pd.DataFrame(dict(Counter(pol_lst)).items(), columns=['word','pol_freq'])
df_pol.set_index('word',inplace=True)

non_pol_lst = flatten_lst(list(df[df["label"]==0]["keywords"]))
df_non_pol = pd.DataFrame(dict(Counter(non_pol_lst)).items(), columns=['word','non_pol_freq'])
df_non_pol.set_index('word',inplace=True)

result = pd.concat([df_non_pol, df_pol], axis=1)
result.reset_index(inplace=True)
result.fillna(0,inplace=True)
result["total_freq"]=result["pol_freq"]+result["non_pol_freq"]
result.sort_values(by=['total_freq'],ignore_index=True,inplace=True,ascending=False)
print(result.head(100))
result.to_csv("/content/drive/MyDrive/datasets/keywords_freq.csv")